In [34]:
import pandas as pd
import numpy as np
import math
from unicodedata import category
import sys
df=pd.read_csv("Train_B_Bayesian.csv")
def normal(x,mu,sigma):
    return (1/(math.sqrt(2*np.pi)*sigma))*np.exp(-(((x-mu)/sigma)**2)/2)

In [35]:
def Bayes(train, val, test, flag):
    posframe=train[train['is_patient']==1]
    negframe=train.drop(posframe.index)
    P={}
    N={}
    for i in train.columns[:-2]:
        P[i]={}
        N[i]={}
        if i=='gender':
            P[i][0]=len(posframe[posframe[i]==0])/len(train)
            N[i][0]=len(negframe[negframe[i]==0])/len(train)
            P[i][1]=len(posframe[posframe[i]==1])/len(train)
            N[i][1]=len(negframe[negframe[i]==1])/len(train)
        else:
            P[i]['mean']=posframe[i].mean()
            P[i]['std']=posframe[i].std()
            N[i]['mean']=negframe[i].mean()
            N[i]['std']=negframe[i].std()
    if(not flag):
        corr=0
        for i in val.index:
            p_prob = 1
            n_prob = 1
            for j in val.columns[:-2]:
                if j=='gender':
                    p_prob*=P[j][val[j][i]]
                    n_prob*=N[j][val[j][i]]
                else:
                    p_prob*=normal(val[j][i],P[j]['mean'],P[j]['std'])*len(posframe)/len(test)
                    n_prob*=normal(val[j][i],N[j]['mean'],N[j]['std'])*len(negframe)/len(test)
            if(p_prob>=n_prob):
                pred=1
            else:
                pred=2
            if(val['is_patient'][i]==pred):
                corr+=1
        return corr/len(val)
    else:
        corr=0
        for i in test.index:
            p_prob = 1
            n_prob = 1
            for j in test.columns[:-2]:
                if j=='gender':
                    p_prob*=P[j][test[j][i]]
                    n_prob*=N[j][test[j][i]]
                else:
                    p_prob*=normal(test[j][i],P[j]['mean'],P[j]['std'])*len(posframe)/len(test)
                    n_prob*=normal(test[j][i],N[j]['mean'],N[j]['std'])*len(negframe)/len(test)
            if(p_prob>=n_prob):
                pred=1
            else:
                pred=2
            if(test['is_patient'][i]==pred):
                corr+=1
        return corr/len(test)

In [36]:
S = df.isnull().sum()
S = S[S!=0]
for i in S.index:
    m = df[i].mean()
    for j in df.index:
        if(np.isnan(df[i][j])):
            df.loc[j,i]=m
            print(i,j,df[i][j])
MAX_OC=0
for j in df.index:
    df.loc[j,'OC']=0
for i in df.columns[:-2]:
    if(df[i].dtype=='object'):
        df[i] = df[i].astype('category').cat.codes
    m=df[i].mean()
    s=df[i].std()
    for j in df.index:
        if(df[i][j]>2*m+5*s):
            df.loc[j,'OC'] += 1
            if(df.loc[j,'OC']>MAX_OC):
                MAX_OC=df.loc[j,'OC']
#print(MAX_OC)
for j in df.index:
    if(df.loc[j,'OC']==MAX_OC):
        df=df.drop(j)


alkphos 209 0.9470639032815197
alkphos 241 0.9470639032815197
alkphos 253 0.9470639032815197
alkphos 312 0.9470639032815197


In [37]:
X=[]
for j in range(40):
    train=df.sample(frac=0.7)
    test=df.drop(train.index)
    test=test.reset_index(drop=True)
    T=np.array_split(train,5)
    acc=[]
    for i in range(5):
        train_5 = train.drop(T[i].index)
        acc.append(Bayes(train_5,T[i],test,0))
    acc=np.array(acc)
    I,max_acc=np.argmax(acc),np.max(acc)
    print("\033[3;36m"+str(I)+"\033[0;m    \033[1;34m"+str(max_acc)+"\033[0;m",file=sys.stderr)
    train_5 = train.drop(T[I].index)
    X.append(Bayes(train_5,T[I],test,1))
    print("\033[3;33m"+str(X[j])+'\033[0;m',file=sys.stderr)
X=np.array(X)
print("\033[5;92m"+str(X.mean())+"    \033[0;m\033[3;91m"+str(X.std())+'\033[0;m')

1    0.7407407407407407
0.6896551724137931
4    0.7037037037037037
0.7126436781609196
3    0.7037037037037037
0.735632183908046
2    0.7160493827160493
0.7011494252873564
2    0.7283950617283951
0.7068965517241379
1    0.7160493827160493
0.632183908045977
3    0.7160493827160493
0.6436781609195402
0    0.7682926829268293
0.6839080459770115
0    0.6951219512195121
0.7298850574712644
2    0.7407407407407407
0.6436781609195402
3    0.7654320987654321
0.6666666666666666
0    0.7317073170731707
0.7413793103448276
0    0.7560975609756098
0.6781609195402298
1    0.7407407407407407
0.7126436781609196
3    0.7160493827160493
0.6494252873563219
3    0.691358024691358
0.7298850574712644
3    0.6790123456790124
0.6724137931034483
0    0.7073170731707317
0.6666666666666666
2    0.7530864197530864
0.6724137931034483
3    0.7901234567901234
0.7011494252873564
1    0.7654320987654321
0.6724137931034483
1    0.654320987654321
0.7586206896551724
2    0.7654320987654321
0.6609195402298851
1    0.74074074

0.6931034482758621    0.037307660614063175


0.7298850574712644
